In [1]:
from data import dataloader
from GPT import GPT, generate
from nltk.tokenize import WordPunctTokenizer
import torch.nn as nn
import torch
import config

Successfully Loaded label dict!
Successfully Loaded label dict!


In [2]:
gpt = GPT(
    vocab_size = config.vocab_size,
    max_len = config.max_len,
    d_model = config.d_model,
    nhead = config.nheads,
    dim_feedforward = config.dim_feedforward,
    num_layers = config.decoder_layers
)
gpt.to(config.device)
# gpt.load_state_dict(torch.load(config.GPT))

GPT(
  (embed): Embedding(7666, 384)
  (DecoderBlock): Decoder(
    (Decoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=384, out_features=384, bias=True)
          )
          (linear1): Linear(in_features=384, out_features=128, bias=True)
          (dropout): Dropout(p=0.2, inplace=False)
          (linear2): Linear(in_features=128, out_features=384, bias=True)
          (norm1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.2, inplace=False)
          (dropout2): Dropout(p=0.2, inplace=False)
        )
        (1): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=384, out_features=384, bias=True)
          )
          (linear1): Lin

In [3]:
loss_func = nn.CrossEntropyLoss()
optim = torch.optim.AdamW(gpt.parameters(), lr=2e-5)
for epoch in range(1):
    gpt.train()
    
    total_train_loss = 0
    for i, (x, y) in enumerate(dataloader):
        print(f"iteration: {i}/{len(dataloader.dataset)/config.batch_size}")
        x = x.to(config.device)
        y = y.to(config.device)
        
        # x丢弃最后一位取头部,丢弃第一位取尾部
        # 实际的任务就是以头部预测尾部
        optim.zero_grad()
        # 模型计算
        # [b, 128] -> [b, 128, 7666]
        pred_y = gpt(x)
        
        # loss_finc([4096 from 32 * 128, 7666], [4096 from 32 * 128])
        loss = loss_func(pred_y.reshape(-1, config.vocab_size), y.reshape(-1))

        loss.backward()
        
        optim.step()
        
        total_train_loss += loss.item()
    
    avg_train_loss = total_train_loss / len(dataloader)
    
    print(f"EPOCH: {epoch}", f"AVG.Loss: {avg_train_loss}")
    total_train_loss = 0
torch.save(gpt.state_dict(), "GPT.pt")

iteration: 0/8177.875
iteration: 1/8177.875
iteration: 2/8177.875
iteration: 3/8177.875
iteration: 4/8177.875
iteration: 5/8177.875
iteration: 6/8177.875
iteration: 7/8177.875
iteration: 8/8177.875
iteration: 9/8177.875
iteration: 10/8177.875
iteration: 11/8177.875
iteration: 12/8177.875
iteration: 13/8177.875
iteration: 14/8177.875
iteration: 15/8177.875
iteration: 16/8177.875
iteration: 17/8177.875
iteration: 18/8177.875
iteration: 19/8177.875
iteration: 20/8177.875
iteration: 21/8177.875
iteration: 22/8177.875
iteration: 23/8177.875
iteration: 24/8177.875
iteration: 25/8177.875
iteration: 26/8177.875
iteration: 27/8177.875
iteration: 28/8177.875
iteration: 29/8177.875
iteration: 30/8177.875
iteration: 31/8177.875
iteration: 32/8177.875
iteration: 33/8177.875
iteration: 34/8177.875
iteration: 35/8177.875
iteration: 36/8177.875
iteration: 37/8177.875
iteration: 38/8177.875
iteration: 39/8177.875
iteration: 40/8177.875
iteration: 41/8177.875
iteration: 42/8177.875
iteration: 43/8177.87

In [ ]:
x = "ROMEO:"
p = generate(gpt, config.tokenizer, x, k=3)
print(" ".join(config.tokenizer.inverse_transform(p.cpu()[0], is_tensor=True)))